### Basic chatbot with prompt template

helps customer to diagnose energy usage anomaly and provide recommendations

In [ ]:
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, trim_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.globals import set_debug

set_debug(True)

load_dotenv()

In [2]:
model = ChatOpenAI(model="gpt-4o-mini")

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


trimmer = trim_messages(
    max_tokens=1000,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an an energy usage anomaly detection assistant.\n
            You are helping a user to detect anomalies in their energy usage.\n
            The user will describe their energy usage and you will help them to detect anomalies.\n
            You will also help the user to identify the causes of the anomalies \n
            and suggest ways to fix them.\n
            """,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = trimmer | prompt | model

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
)

config = {"configurable": {"session_id": "abc20"}}

In [ ]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi my smart meter says I owe 1 trillion pounds")]},
    config=config,
)

response.content